In [1]:
from __future__ import division
import nltk, random
from nltk.probability import DictionaryProbDist as D
import re
from collections import defaultdict, Counter
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
import nltk, random
# from sklearn.model_selection import train_test_split

In [2]:
def leskOverlap(senseDef, target):
    wnidCount = [ (wncat, tf, word, len(DF[word])+1) for word in senseDef \
                                                    for wncat, tf in TF[word].items() \
                                                    if wncat in target.values() ]
    res = sorted( [ (word, tf*int(1000/df)) for wnid, tf, word, df in wnidCount], key = lambda x: -x[1])[:5]
    feature = {}
#         print(res)
    for i in res:
        feature[i[0]] = i[1]
#     print(feature)
        
    return feature

In [3]:
def words(text): return re.findall(r'\w+', text.lower())
    
TF = defaultdict(lambda: defaultdict(lambda: 0))
DF = defaultdict(lambda: [])

def wnTag(pos): return {'noun': 'n', 'verb': 'v', 'adjective': 'a', 'adverb': 'r'}[pos]

def trainLesk():
    training_x = []
    training_y = []
    features = []
    training = [  line.strip().split('\t') for line in open('wn.in.evp.cat.txt', 'r') if line.strip() != '' ]
#     training = training[:100]
#     training, testing = train_test_split(training,test_size=0.2)
#     print(training[0])
    def isHead(head, word, tag):
        try:
            return lmtzr.lemmatize(word, tag) == head
        except:
            return False
            
    # zucchini-n-2	vegetable.n.01	zucchini courgette||small cucumber-shaped vegetable marrow; typically dark green||	
    # {'zucchini-n-1': 'vine.n.01', 'zucchini-n-2': 'vegetable.n.01'}        
    for wnid, wncat, senseDef, target in training:
#         print(senseDef)
        Tf = {}
        head, pos = wnid.split('-')[:2]
        for word in words(senseDef):
            Tf[word] = True
#         print(Tf)
#         training_x.append(Tf)
#         training_y.append(wncat)
        
            if word != head and not isHead(head, word, pos):
#                 f = gender_features(word)
#                 print(type(f),f)
#                 Tf.append(f)
#         training_data.append(Tf)
#         training_label.append(wncat)
#                 print(word)
                TF[word][wncat] += 1
                DF[word] += [] if wncat in DF[word] else [wncat]
#                 print(TF[word][wncat])
#             features.append((Tf,wncat))
            

    for evpid, wncat, senseDef, target in training:
        features.append((leskOverlap(words(senseDef), eval(target)),wncat))
        #head, pos = evpid.split('-')[:2]
#         print( '%s\t%s\t%s' %(evpid, leskOverlap(words(senseDef), eval(target)), senseDef.split('||')[0] ) )
#             Tf[word] = True
    
            
#     print(DF["abandon"]["unconcerned.a.01"])
    return features

In [ ]:
features = trainLesk()


In [ ]:
print(features[0])

In [ ]:
def LG_gender(train_set, test_set):
    print('== SkLearn MaxEnt ==')
    from nltk.classify import SklearnClassifier 
    from sklearn.linear_model import LogisticRegression
    sklearn_classifier = SklearnClassifier(LogisticRegression(C=10e5)).train(train_set)
#     print(sklearn_classifier.prob_classify(gender_features('mark'))._prob_dict)
    print(nltk.classify.accuracy(sklearn_classifier, test_set))
    return sklearn_classifier

def ME_gender(train_set, test_set):
    print('== NLTK MaxEnt ==')
    from nltk.classify import MaxentClassifier
    nltk_classifier = MaxentClassifier.train(train_set, nltk.classify.MaxentClassifier.ALGORITHMS[0])
#     print(nltk_classifier.prob_classify(gender_features('mark'))._prob_dict)
    print(nltk.classify.accuracy(nltk_classifier, test_set))

In [ ]:
split_point = len(features)*9//10
train_set, test_set = features[:split_point], features[split_point:]
classifier = LG_gender(train_set, test_set)
# ME_gender(train_set, test_set)